In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [3]:
data = pd.read_feather('database/base.feather')
data

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Region,...,OMonth,OPeriod,Order Date Month,SYear,SMonth,SPeriod,Ship Date Month,Delivery,Price,Benefit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,130.98,1
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,243.98,1
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,Los Angeles,California,West,...,6,30,2016-06-01,2016,6,30,2016-06-01,4,7.31,1
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,191.52,0
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,11.18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,Miami,Florida,South,...,1,1,2014-01-01,2014,1,1,2014-01-01,2,8.42,1
9990,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,45.98,1
9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,129.29,1
9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,7.40,1


In [5]:
def deep_learning_dnn(df_dl, dep_var, classes):
    # Separa a variável dependente das demais
    deep_feat = df_dl.drop(columns=[dep_var], axis=1)
    deep_label = df_dl[dep_var]
    # Verifica os tipos das variáveis
    categorical_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) == 2 or deep_feat[col].dtype == 'O']
    continuous_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) > 2 and (deep_feat[col].dtype == 'int64' or deep_feat[col].dtype == 'float64')]
    # Verifica as colunas para normalização - as demais serão discretizadas - Função Bucketize do Tensor Flow
    cols_to_scale = continuous_columns[:]
    #cols_to_scale.remove('meses')
    # Ajusta as bases de treino e de teste
    XX_T = df_dl.drop(columns=[dep_var], axis=1)
    XX_t = df_dl.drop(columns=[dep_var], axis=1)
    yy_T = df_dl[dep_var]
    yy_t = df_dl[dep_var]
    # Normaliza as variáveis nas bases de treino e teste
    scaler = StandardScaler()
    XX_T.loc[:, cols_to_scale] = scaler.fit_transform(XX_T.loc[:, cols_to_scale]) ## Treinamento
    XX_t.loc[:, cols_to_scale] = scaler.fit_transform(XX_t.loc[:, cols_to_scale]) ## Teste
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_object_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_hash_bucket(key=col, hash_bucket_size=1000), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df_dl[col].dtype == 'O']
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_integer_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_identity(key=col, num_buckets=2), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df[col].dtype=='int64']
    continuous_feat_cols = [tf.feature_column.numeric_column(key=col) for col in continuous_columns] 
    feat_cols = categorical_object_feat_cols + \
                categorical_integer_feat_cols + \
                continuous_feat_cols
    # Rotina de DNN (Deep Neural Network)
    input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_T, yy_T, batch_size=50, num_epochs=1000, shuffle=True)
    pred_input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_t, batch_size=50, shuffle=False)
    DNN_model = tf.estimator.DNNClassifier(hidden_units=[10, 10, 10], feature_columns=feat_cols, n_classes=classes)
    DNN_model.train(input_fn=input_fun, steps=5000)
    # Resgata os resultados da DNN
    predictions = DNN_model.predict(pred_input_fun)
    pred = list(predictions)
    return pred

In [6]:
## Aula 14 - Probabilidade
dimensao = 'State'
medidas = ['Sales', 'Quantity', 'Profit']
grupo = data.groupby(dimensao)[medidas].mean().reset_index()
grupo['Benefit'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)
grupo = grupo.set_index(dimensao)

In [ ]:
probabilidade = deep_learning_dnn(grupo, 'Benefit', 2)
probabilidade_classe = []
for i in range(len(probabilidade)):
    probabilidade_classe.append(probabilidade[i]["class_ids"][0])
probabilidade_prob0 = []
for i in range(len(probabilidade)):
    probabilidade_prob0.append(probabilidade[i]["probabilities"][0])
probabilidade_prob1 = []
for i in range(len(probabilidade)):
    probabilidade_prob1.append(probabilidade[i]["probabilities"][1]) 
grupo['dl_classe'] = probabilidade_classe
grupo['lucro_0'] = probabilidade_prob0
grupo['lucro_1'] = probabilidade_prob1
grupo = grupo.reset_index()
grupo

In [ ]:
grupo.to_feather('database/probabilidade_estado.feather')